In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sunpy
from sunpy.map import Map
from sunpy.instr.aia import aiaprep
# Show original image?
# Load SSW processed image
# Show differences between processed images and original
# Show difference between processed images

In [ ]:
fig = plt.figure(figsize=(15, 8))

# Load original image
origim = Map("aia171_2014-01-11T00_00.fits")
print origim.scale, origim.shape
fig.add_subplot(1, 2, 1)
origim.plot()
plt.title('Max: {:.2f}, Mean: {:.2f}, Min: {:.2f}'.format(origim.max(), origim.mean(), origim.min()))
plt.colorbar(orientation='horizontal')

# Process original image with SunPy
pypreppedim = aiaprep(origim)
fig.add_subplot(1, 2, 2)
pypreppedim.plot()
plt.title('Max: {:.2f}, Mean: {:.2f}, Min: {:.2f}'.format(pypreppedim.max(), pypreppedim.mean(), pypreppedim.min()))
plt.colorbar(orientation='horizontal')
plt.show()

In [ ]:
diffim1 = Map(pypreppedim.data.copy(), pypreppedim.meta.copy())
diffim1.data -= origim.data
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 30))
plt.sca(ax[0])
meandiff = diffim1.mean()
stddiff = diffim1.std()
diffim1.plot(cmap='coolwarm', vmin=meandiff-(2*stddiff), vmax=meandiff+(2*stddiff))
plt.title('Min: {:.2f}, Max{:.2f}\nMean: {:.2f}, Std Dev: {:.2f}'.format(diffim1.max(), diffim1.min(), meandiff, stddiff))
plt.colorbar(orientation='horizontal')
plt.sca(ax[1])
plt.hist(diffim1.data.flatten(), range=(meandiff-(2*stddiff), meandiff+(2*stddiff)), bins=50)
plt.show()

In [ ]:
print origim.processing_level, pypreppedim.processing_level
print pypreppedim.scale, pypreppedim.shape
print np.nanmin(pypreppedim.data), np.nanmean(pypreppedim.data), np.nanmax(pypreppedim.data)
print pypreppedim.min(), pypreppedim.mean(), pypreppedim.max()
print origim.center, origim.reference_coordinate, origim.reference_pixel
print pypreppedim.center, pypreppedim.reference_coordinate, pypreppedim.reference_pixel